In [47]:
# import QGRID para mejor visualizacion de los datos
# los pptos sin abonos se deben cancelar en ClinicCloud, cosa q no generen alertas. O q el paciente tenga saldo a favor
# Debemos Generar un archivo identificandolos. Las ff´s deben tener algun ingreso de dinero para no cancelarlas.
# Los pptos aceptados hoy pueden haber sido elaborados en una fecha anterior. Su fecha se debe actualizar a la de hoy, 
# ya sea en ClinicCloud o en el reporte (mejor - 'fecha_inicio')

import numpy as np
import pandas as pd
import time

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', -1)

In [48]:
archivo_PPE = 'Produccion_pendiente_exigible_con_tratamientos_Dentix_Calle_90.csv'
archivo_crono = 'crono_Dentix_Calle_90_1.csv'

In [49]:
crono_df = pd.read_csv(archivo_crono, encoding = 'unicode_escape', sep=',')
PPE = pd.read_csv(archivo_PPE, encoding = 'unicode_escape', sep=',')

#Eliminación de Puntos y Revisiones del PPE
PPE.set_index('Servicio', inplace=True)
PPE.drop(index='Puntos y Revisiones.  ',inplace=True)
PPE.reset_index(inplace=True)
PPE['duplicated'] = PPE.duplicated()
PPE.set_index('duplicated',inplace=True)
PPE.drop(True,inplace=True)
PPE.reset_index(inplace=True)
PPE.drop(columns=['duplicated'],inplace=True)
PPE

,Servicio,Clínica,Fecha Pres.,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor,Especialidad,Explicado,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod
0,DETARTRAJE POR ARCADA.,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,97520,195000,100%,0.00%,0,195000,06/09/2018,Sin Cita Producida
1,PROFILAXIS. Pieza: 11,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,40000,40000,100%,0.00%,0,40000,06/09/2018,Sin Cita Producida
2,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 45,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,79000,79000,100%,0.00%,0,79000,06/09/2018,Sin Cita Producida
3,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 16,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,79000,79000,100%,0.00%,0,79000,06/09/2018,Sin Cita Producida
4,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 48,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,79000,79000,100%,0.00%,0,79000,06/09/2018,Sin Cita Producida
5,RESINA FOTOPOLIMERIZADO 3 SUP POSTERIOR O POSTERIOR. Pieza: 26,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,116500,116500,100%,0.00%,0,116500,06/09/2018,Sin Cita Producida
6,EXODONCIA DE RESTOS RADICULARES. Pieza: 15,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,CIRUJIA ORAL Y MAXILOFACIAL,SANDRA PATRICIA GONZALEZ TOVAR,249100,249100,100%,0.00%,0,249100,06/09/2018,Sin Cita Producida
7,PROTESIS TEMPORAL 1/3 DIENTES.,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,296800,162400,27.36%,0.00%,0,593600,06/09/2018,Sin Cita Producida
8,TAC 2 Arcadas. Pieza: 31,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,RADIOLOGÍA Y PRUEBAS DIAGNÓSTICAS,SANDRA PATRICIA GONZALEZ TOVAR,207000,0,0.00%,0.00%,0,207000,06/09/2018,Sin Cita Producida
9,CITA TALLADO SELECTIVO X PIEZA. Pieza: 47,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OCLUSION Y DISFUNCION TEMPOROMANDIBULAR,SANDRA PATRICIA GONZALEZ TOVAR,118720,0,0.00%,0.00%,0,118700,06/09/2018,Sin Cita Producida


In [50]:
print('REGISTROS CRONOGRAMA:',crono_df.count()[0])
print('REGISTROS PPE:',PPE.count()[0])

REGISTROS CRONOGRAMA: 291
REGISTROS PPE: 375


In [51]:
#CRUCE PPE - CRONO: en este punto revisar que cada ttmto del PPE tenga su cronograma. Sinó, sacar los pptos q faltan por crono.

pptos_faltantes = pd.merge(PPE,crono_df,how='left',left_on=['Clínica','Nº Historia','Presupuesto','Servicio'], right_on=['Clínica','Nº Historia','Presupuesto','Servicio'])
pptos_faltantes['Semana'].fillna(0,inplace=True)
pptos_faltantes.set_index('Semana',inplace=True)
pptos_faltantes = pptos_faltantes.loc[0]

pptos_faltantes

,Servicio,Clínica,Fecha Pres._x,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor_x,Especialidad,Explicado_x,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod,Fecha Pres._y,Cantidad Presupuestada,Doctor_y,Explicado_y,ttmto_homol,pieza,fecha_inicio
Semana,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,DETARTRAJE POR ARCADA.,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,97520,195000,100%,0.00%,0,195000,06/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,PROFILAXIS. Pieza: 11,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,40000,40000,100%,0.00%,0,40000,06/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 45,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,79000,79000,100%,0.00%,0,79000,06/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 16,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,79000,79000,100%,0.00%,0,79000,06/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 48,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,79000,79000,100%,0.00%,0,79000,06/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RESINA FOTOPOLIMERIZADO 3 SUP POSTERIOR O POSTERIOR. Pieza: 26,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,116500,116500,100%,0.00%,0,116500,06/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,EXODONCIA DE RESTOS RADICULARES. Pieza: 15,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,CIRUJIA ORAL Y MAXILOFACIAL,SANDRA PATRICIA GONZALEZ TOVAR,249100,249100,100%,0.00%,0,249100,06/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,PROTESIS TEMPORAL 1/3 DIENTES.,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,296800,162400,27.36%,0.00%,0,593600,06/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,TAC 2 Arcadas. Pieza: 31,Dentix Calle 90,05/09/2018,BG058120,Nº 4237 - Edison Diaz,0,3134327054,MARIA FERNANDA SUAREZ BECERRA,RADIOLOGÍA Y PRUEBAS DIAGNÓSTICAS,SANDRA PATRICIA GONZALEZ TOVAR,207000,0,0.00%,0.00%,0,207000,06/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
pptos_faltantes.reset_index(inplace=True)
pptos_faltantes['Cantidad Presupuestada'] = (pptos_faltantes['Producidos']+pptos_faltantes['Pendientes Producción'])/pptos_faltantes['Importe Pres.']
pptos_faltantes['Cantidad Presupuestada'] = np.ceil(pptos_faltantes['Cantidad Presupuestada'])
pptos_faltantes['Cantidad Presupuestada'].fillna(1,inplace=True) #las CSI traen vacío el campo 'Pendientes Produccion'.
# en realidad parece que las unicas cantidades presupuestadas q son necesarias son las de Mensualidades de ortodoncia.
# Para CSI, el %Realizado es el que nos sirve para calcular Cantidad Producida.
# la Cantidad Pendiente se puede utilizar para saber la fecha de colocación de un implante si hay ortodoncia.

pptos_faltantes['ttmto_homol'] = pptos_faltantes['Servicio'].str.slice(0,-13)
pptos_faltantes['ttmto_homol'] = pptos_faltantes['ttmto_homol'].str.strip()
pptos_faltantes['pieza'] = pptos_faltantes['Servicio'].str.slice(-2)
pptos_faltantes['pieza'] = pptos_faltantes['pieza'].str.strip()
pptos_faltantes['fecha_inicio'] = time.strftime("%d/%m/%Y")

pptos_faltantes = pptos_faltantes[['Clínica', 'Fecha Pres._x', 'Nº Historia', 'Presupuesto', 
                                   'Servicio', 'Semana', 'Cantidad Presupuestada','Doctor_x', 
                                   'Explicado_x', 'ttmto_homol','pieza', 'fecha_inicio']]

print('Numero de tratamientos faltantes por crono:',pptos_faltantes.count()[0])
pptos_faltantes.to_csv('pptos_faltantes.csv',index = False)

pptos_faltantes.set_index(['Clínica','Nº Historia','Servicio'], inplace=True)
pptos_faltantes

Numero de tratamientos faltantes por crono: 88


Fecha Pres._x  \
Clínica         Nº Historia Servicio                                                                                              
Dentix Calle 90 BG058120    DETARTRAJE POR ARCADA.                                                                 05/09/2018     
                            PROFILAXIS.   Pieza: 11                                                                05/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 45                        05/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 16                        05/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 48                        05/09/2018     
                            RESINA FOTOPOLIMERIZADO 3 SUP POSTERIOR O POSTERIOR.   Pieza: 26                       05/09/2018     
                            EXODONCIA DE RESTOS RADICULARES.   Pieza: 15                                           05/09/2018     
                            PROTESIS TEMPORAL 1/3 DIENTES.                                                         05/09/2018     
                            TAC 2 Arcadas.   Pieza: 31                                                             05/09/2018     
                            CITA TALLADO SELECTIVO X PIEZA.   Pieza: 47                                            05/09/2018     
                            IMPLANTE.   Pieza: 14                                                                  05/09/2018     
                            IMPLANTE.   Pieza: 15                                                                  05/09/2018     
                            IMPLANTE.   Pieza: 46                                                                  05/09/2018     
                            CORONA SOBRE IMPLANTE.   Pieza: 14                                                     05/09/2018     
                            CORONA SOBRE IMPLANTE.   Pieza: 15                                                     05/09/2018     
                            CORONA SOBRE IMPLANTE.   Pieza: 46                                                     05/09/2018     
                BG058121    EXODONCIA RESTO RADICULAR QUIRURGICO.   Pieza: 22                                      05/09/2018     
                            PROTESIS TEMPORAL 1/3 DIENTES.                                                         05/09/2018     
                            IMPLANTE.   Pieza: 22                                                                  05/09/2018     
                            CORONA SOBRE IMPLANTE ZIRCONIO.   Pieza: 22                                            05/09/2018     
                            TAC 1 Arcada.   Pieza: 22                                                              05/09/2018     
                            DETARTRAJE POR ARCADA.                                                                 05/09/2018     
                            PROFILAXIS.   Pieza: 11                                                                05/09/2018     
                            BLANQUEAMIENTO DENTAL 2.   Pieza: 11                                                   05/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 27                        05/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 42                        05/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 47                        05/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 21                        05/09/2018     
                            PLACA NEUROMIORELAJANTE ACTIVADA.   Pieza: 11                                          05/09/2018     
                BG052725    PLACA NEUROMIORELAJANTE A